In [4]:

import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interactive_output
from IPython.display import display, Markdown

# Load dataset
pivot_df = pd.read_csv('cleaned_gender_pivot.csv')  # Assumes this file is available


In [5]:

summary_html = widgets.HTML(
    value=f"<h3>Total Records: {len(pivot_df)}</h3>"
)
display(Markdown("### Dataset Summary"))
display(summary_html)


### Dataset Summary

HTML(value='<h3>Total Records: 216</h3>')

In [6]:

# Field filter as ToggleButtons
field_options = ['All Fields'] + sorted(pivot_df['Field of Study'].unique().tolist())
field_buttons = widgets.ToggleButtons(
    options=field_options,
    value='All Fields',
    description='Field:'
)

# Region filter
region_options = ['All Regions'] + sorted(pivot_df['HSE Health Regions'].unique().tolist())
region_buttons = widgets.ToggleButtons(
    options=region_options,
    value='All Regions',
    description='Region:'
)

# Year filter
year_mode = widgets.ToggleButtons(
    options=['All Years', 'Year Range'],
    value='All Years',
    description='Year View:'
)
year_slider = widgets.IntRangeSlider(
    value=[pivot_df['Graduation Year'].min(), pivot_df['Graduation Year'].max()],
    min=pivot_df['Graduation Year'].min(),
    max=pivot_df['Graduation Year'].max(),
    step=1,
    description='Years:',
    continuous_update=False
)
year_container = widgets.Output()
def update_year_widget(change):
    year_container.clear_output()
    with year_container:
        display(year_slider if change['new'] == 'Year Range' else widgets.HTML(""))

year_mode.observe(update_year_widget, names='value')
update_year_widget({'new': year_mode.value})

display(Markdown("### Filters"))
display(widgets.VBox([field_buttons, region_buttons, year_mode, year_container]))


### Filters

In [7]:

def update_bar_chart(field, region, view_mode, year_range):
    df = pivot_df.copy()
    if field != 'All Fields':
        df = df[df['Field of Study'] == field]
    if region != 'All Regions':
        df = df[df['HSE Health Regions'] == region]
    if view_mode == 'Year Range':
        start, end = year_range
        df = df[(df['Graduation Year'] >= start) & (df['Graduation Year'] <= end)]
    grouped = df.groupby('HSE Health Regions')[['Female', 'Male']].sum().reset_index()
    fig = px.bar(grouped, x='HSE Health Regions', y=['Female', 'Male'], barmode='group',
                 title='Graduate Counts by Gender and Region',
                 labels={'value': 'Graduates', 'HSE Health Regions': 'Region'}, height=500)
    fig.show()

bar_output = interactive_output(update_bar_chart, {
    'field': field_buttons,
    'region': region_buttons,
    'view_mode': year_mode,
    'year_range': year_slider
})

display(Markdown("### Chart 1: Gender Breakdown by Region"))
display(bar_output)


### Chart 1: Gender Breakdown by Region

Output()

In [8]:

def update_donut_chart(field, region, view_mode, year_range):
    df = pivot_df.copy()
    if field != 'All Fields':
        df = df[df['Field of Study'] == field]
    if region != 'All Regions':
        df = df[df['HSE Health Regions'] == region]
    if view_mode == 'Year Range':
        start, end = year_range
        df = df[(df['Graduation Year'] >= start) & (df['Graduation Year'] <= end)]
    gender_totals = df[['Female', 'Male']].sum()
    fig = px.pie(values=gender_totals, names=gender_totals.index, hole=0.5,
                 title='Gender Share of Graduates',
                 labels={'label': 'Gender', 'value': 'Count'})
    fig.update_traces(textposition='inside', textinfo='label+percent+value')
    fig.show()

donut_output = interactive_output(update_donut_chart, {
    'field': field_buttons,
    'region': region_buttons,
    'view_mode': year_mode,
    'year_range': year_slider
})

display(Markdown("### Chart 2: Gender Share (Donut Chart)"))
display(donut_output)


### Chart 2: Gender Share (Donut Chart)

Output()

In [9]:

def update_box_chart(field, region, view_mode, year_range):
    df = pivot_df.copy()
    if field != 'All Fields':
        df = df[df['Field of Study'] == field]
    if region != 'All Regions':
        df = df[df['HSE Health Regions'] == region]
    if view_mode == 'Year Range':
        start, end = year_range
        df = df[(df['Graduation Year'] >= start) & (df['Graduation Year'] <= end)]
    df_melt = df.melt(id_vars=['Graduation Year', 'HSE Health Regions'], value_vars=['Male', 'Female'],
                      var_name='Gender', value_name='Graduates')
    fig = px.box(df_melt, x='HSE Health Regions', y='Graduates', color='Gender',
                 points='all', hover_data=['Graduation Year'],
                 title='Outlier Detection by Region and Gender')
    fig.update_traces(marker=dict(size=6))
    fig.show()

box_output = interactive_output(update_box_chart, {
    'field': field_buttons,
    'region': region_buttons,
    'view_mode': year_mode,
    'year_range': year_slider
})

display(Markdown("### Chart 3: Outlier Detection (Box Plot)"))
display(box_output)


### Chart 3: Outlier Detection (Box Plot)

Output()